<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/PVT/parameter_database2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install neqsim
!pip install wget

# NeqSim using your own property database

In this notebook, we will demonstrate how to utilize your own customized parameter database within the NeqSim thermodynamic framework. This functionality allows you to:

* **Extend NeqSim's capabilities:** By incorporating new components and their associated properties, you can model systems beyond the default database.
* **Improve accuracy:**  Refine existing component parameters or introduce new interaction parameters to enhance the precision of your simulations.
* **Tailor to specific needs:** Create databases tailored to your particular research or industrial applications.

We will guide you through the process of:

1. Accessing and modifying NeqSim's database tables.
2. Adding new components with comprehensive property data.
3. Defining interaction parameters for mixtures.
4. Updating existing component parameters.
5. Using the `Chemicals` package to gather property data for new components.

By the end of this notebook, you will have the knowledge to effectively leverage your own property databases within NeqSim, opening up a wider range of thermodynamic modeling possibilities.

# Further reading
See:
https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/PVT/parameter_database.ipynb


and

https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/readproperties.ipynb

# NeqSim parameter database
NeqSim reads pure and mixture component parmaters from a built in database.

The database tables used are stored in this folder:

https://github.com/equinor/neqsim/tree/master/src/main/resources/data

The two main tables are the COMP.csv file with pure component parameters and the INTER.csv file with the binary interaction parameters.

THe other tables are also important but used for various specialized calculations.

# Use your own database

You can establish your own database table by storing your parameters in a file with the same format as, for example, COMP.csv, and set that table to be used by NeqSim.

In the following example, we will show how this can be done.

We first download the COMP and INTER tables from GitHub and then rename them to COMP_user and INTER_user. The tables can then be changed or extended by the user. These files should be stored on your local drive and read into neqsim every time you run the program.

After this is done, we write the tables back to NeqSim, and the new tables will be used as the basis for calculations.

In [10]:
# we start by downloading the COMP and INTER tables

%%capture
!wget https://raw.githubusercontent.com/equinor/neqsim/master/src/main/resources/data/COMP.csv
!wget https://raw.githubusercontent.com/equinor/neqsim/master/src/main/resources/data/INTER.csv

import os

# Rename the COMP table
os.rename('COMP.csv', 'COMP_user.csv')

# Rename the INTER table
os.rename('INTER.csv', 'INTER_user.csv')

!chmod 644 /content/COMP_user.csv

Further, we will modify the tables and add components directly in the csv text file (not done here).

We will then write it back to NeqSim by overwriting the COMP or INTER database.

In [24]:
from neqsim import jneqsim as neqsim
from neqsim.thermo import TPflash, printFrame

# Replace the default 'COMP' table in the NeqSim database with the user-defined 'COMP_user.csv' table
# located at '/content/COMP_user.csv'.
neqsim.util.database.NeqSimDataBase.replaceTable('COMP', '/content/COMP_user.csv')
# Replace the default 'INTER' table in the NeqSim database with the user-defined 'INTER_user.csv' table
# located at '/content/INTER_user.csv'.
neqsim.util.database.NeqSimDataBase.replaceTable('INTER', '/content/INTER_user.csv')

#and then use the new component and table
from neqsim.thermo import fluid

fluid1 = fluid('srk')
fluid1.setTemperature(20.0, 'C')
fluid1.setPressure(1.1, 'bara')
fluid1.addComponent('methane', 1.0)
TPflash(fluid1)
printFrame(fluid1)

| 0                    | 1         | 2          | 3   | 4   | 5   | 6               |
|:---------------------|:----------|:-----------|:----|:----|:----|:----------------|
|                      | total     | GAS        |     |     |     |                 |
| methane              | 1E0       | 1E0        |     |     |     | [mole fraction] |
|                      |           |            |     |     |     |                 |
| Density              |           | 7.25498E-1 |     |     |     | kg/m3           |
| Phase Fraction       |           | 1E0        |     |     |     | [mole fraction] |
| Molar Mass           | 1.6043E-2 | 1.6043E-2  |     |     |     | kg/mol          |
| Z factor             |           | 9.97971E-1 |     |     |     | [-]             |
| Heat Capacity (Cp)   |           | 2.22537E0  |     |     |     | kJ/kgK          |
| Heat Capacity (Cv)   |           | 1.70117E0  |     |     |     | kJ/kgK          |
| Speed of Sound       |           | 4.44916E2  |     

# Changing parameters from program code
THe alternative way of changing parameters in neqsim, is to do it derectly in the program code.

We then set parameters like Tc, Pc and ecentric factors directly on the fluid.

THis is illustrated in the folloing code.

In [30]:
from neqsim.thermo import fluid, TPflash, printFrame

# Create a fluid object using the SRK equation of state
fluid1 = fluid('srk')

# Add methane to the fluid with a mole fraction of 1.0
fluid1.addComponent('methane', 1.0)

# Set the fluid temperature and pressure
fluid1.setTemperature(20.0, 'C')
fluid1.setPressure(1.1, 'bara')

for phase_number in range(fluid1.getMaxNumberOfPhases()):
  # Get the component (methane) and modify its properties
  methane = fluid1.getPhase(phase_number).getComponent('methane')

  # Change critical temperature (Tc) to 195.0 K
  methane.setTC(195.0, 'K')
  # Change critical pressure (Pc) to 50.0 bara
  methane.setPC(50.0, 'bara')
  # Change acentric factor (omega) to 0.011
  methane.setAcentricFactor(0.011)

# Perform a two-phase flash calculation on the fluid
TPflash(fluid1)

# Print the results of the flash calculation in a formatted table
printFrame(fluid1)

| 0                    | 1         | 2          | 3   | 4   | 5   | 6               |
|:---------------------|:----------|:-----------|:----|:----|:----|:----------------|
|                      | total     | GAS        |     |     |     |                 |
| methane              | 1E0       | 1E0        |     |     |     | [mole fraction] |
|                      |           |            |     |     |     |                 |
| Density              |           | 7.25504E-1 |     |     |     | kg/m3           |
| Phase Fraction       |           | 1E0        |     |     |     | [mole fraction] |
| Molar Mass           | 1.6043E-2 | 1.6043E-2  |     |     |     | kg/mol          |
| Z factor             |           | 9.97963E-1 |     |     |     | [-]             |
| Heat Capacity (Cp)   |           | 2.22514E0  |     |     |     | kJ/kgK          |
| Heat Capacity (Cv)   |           | 1.70112E0  |     |     |     | kJ/kgK          |
| Speed of Sound       |           | 4.44896E2  |     